# Qualtrics integration and pipeline
Demonstration of how we will generate archetypes from the *Qualtrics* responses both for the current car as well as for each vintage (i.e. archetype defined by `vehicle_type` and `fuel_type`)

## Qualtrics

In [ ]:
import os

from QualtricsAPI.Setup import Credentials

from QualtricsAPI import Responses

In [ ]:
TOKEN = os.environ.get('QUALTRICS_API_KEY')
DATA_CENTER = os.environ.get('QUALTRICS_BASE_URL').split('.')[0]

In [ ]:
Credentials().qualtrics_api_credentials(token=TOKEN, data_center=DATA_CENTER)

In [ ]:
r = Responses()
survey_id = 'SV_6u4YzY24OAfhWfQ'
df_pre_study = r.get_survey_responses('SV_6u4YzY24OAfhWfQ')

In [ ]:
df_pre_study.head()

## Workflow

In [ ]:
import tcsscraper.scrape as tcs

from tcsscraper.helper import Car

In [ ]:
Car.vehicle_classes

In [ ]:
Car.fuel_types

In [ ]:
# car = Car('Luxusklasse', 'Elektro', 5)

In [ ]:
car = Car('Luxusklasse', 'Benzin', 10)
print(car)

In [ ]:
similar_cars = tcs.get_similar_cars(car, km=20e3, canton='ZH', buffer=5, verbose=True)

In [ ]:
len(similar_cars)

In [ ]:
def remove_unit(string):
    """Removes unit from string like 17'000 CHF/year -> float(17e3)

    Args:
        string (str): string with unit
    """
    number, unit = string.split(' ')
    number = number.replace("'", '')
    return float(number)

In [ ]:
import pandas as pd

def generate_archetype(similar_cars):
    """Generates an archetypical car from a list of similar cars

    Args:
        similar_cars (list of dict): returned by get_similar_cars()
    """
    car_attributes = []
    for c in similar_cars:
        costs = c['costs']
        attrs = {
            'fix_cost': remove_unit(costs['Fixe Kosten']),
            'variable_cost': remove_unit(costs['Variable Kosten']),
            'cost_per_km': remove_unit(costs['Kilometerkosten'])
        }
        car_attributes.append(attrs)

    car_attributes = pd.DataFrame(car_attributes)

    archetype = dict(car_attributes.mean())

    return archetype
    

In [ ]:
archetype = generate_archetype(similar_cars)
archetype

## Pipeline

In [ ]:
## generate data.frame with variables to feed to get_similar_cars
df_qualt = pd.DataFrame({
    'vehicle_type': ['SUV S', 'Luxusklasse', 'Mittelklasse'],
    'fuel_type': ['Benzin', 'Benzin', 'Benzin'],
    'fuel_consumption': [3, 7, 2],
    'km': [15e3, 20e3, 22e3],
    'canton': ['VD', 'AI', 'ZG']
})

df_qualt.head()

In [23]:
def helper(row, max=5, verbose=True):
    car = Car(row['vehicle_type'], row['fuel_type'], row['fuel_consumption'])
    init = 1
    if verbose:
        print('---\n{}\n---'.format(car))
    while(init < max+1):
        try:
            sc = tcs.get_similar_cars(car, row['km'], row['canton'], buffer=init, verbose=verbose)
        except Exception:
            init += 1
            continue
        return sc
    return None

In [24]:

df_qualt['similar_cars'] = df_qualt.apply(lambda row: helper(row), axis=1)

---
vehicle_type:     SUV S
fuel_type:        Benzin
fuel_consumption: 3
---
Extracted Seat Arona 1.0 EcoTSI 110 Style
Extracted Hyundai Bayon 1.0 T-GDi Origo DCT
Extracted Skoda Kamiq 1.0 TSI 115 Ambition
Extracted Hyundai Bayon 1.0 T-GDi Pica
Extracted Skoda Kamiq 1.0 TSI Ambition
Extracted VW T-Cross 1.0 TSI 110 Life
Extracted Skoda Kamiq 1.5 TSI 150 Style
Extracted Kia Stonic 1.0 T-GDi Power
---
vehicle_type:     Luxusklasse
fuel_type:        Benzin
fuel_consumption: 7
---
Extracted BMW 7 Limousine 740Li xDrive
Extracted BMW 8 Coupé 840i xDrive
Extracted BMW 8 Gran Coupé 840i xDrive
Extracted BMW 8 Cabriolet 840i xDrive
---
vehicle_type:     Mittelklasse
fuel_type:        Benzin
fuel_consumption: 2
---
Extracted Seat Leon Sportstourer 1.0 TSI Style
Extracted Seat Leon Sportstourer 1.5 EcoTSI Style 130
Extracted Seat Leon Sportstourer 1.5 TSI Style 150


In [28]:
generate_archetype(df_qualt['similar_cars'][0])

{'fix_cost': 5228.625,
 'variable_cost': 3492.125,
 'cost_per_km': 0.5900000000000001}

In [30]:
## generate archetypes
df_qualt['archetpye'] = df_qualt.apply(lambda row: generate_archetype(row['similar_cars']), axis=1)
df_qualt['archetpye']

0    {'fix_cost': 5228.625, 'variable_cost': 3492.1...
1    {'fix_cost': 15525.0, 'variable_cost': 11557.0...
2    {'fix_cost': 6178.333333333333, 'variable_cost...
Name: archetpye, dtype: object

## TODO
- `fuel_consumption` with `fuel_type='electric'` does not work